## Carregando dataset

---


In [0]:
import pandas as pd

df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')
df.describe()

,nome,descricao,categoria
count,4080,2916,4080
unique,3696,2460,4
top,Bonecos Brinquedo Vingadores 30 Cm Com Luz E...,JOGO ORIGINAL. NOVO. LACRADO. PRONTA ENTREGA. ...,maquiagem
freq,20,39,1020


In [0]:
df.dropna(inplace=True)

df["texto"] = df['nome'] + " " + df['descricao']

df.texto[0]

' O Hobbit - 7ª Ed. 2013  Produto NovoBilbo Bolseiro é um hobbit que leva uma vida confortável e sem ambições. Mas seu contentamento é perturbado quando Gandalf, o mago, e uma companhia de anões batem à sua porta e levam-no para uma expedição. Eles têm um plano para roubar o tesouro guardado por Smaug, o Magnífico, um grande e perigoso dragão. Bilbo reluta muito em participar da aventura, mas acaba surpreendendo até a si mesmo com sua esperteza e sua habilidade como ladrão!CaracterísticasAutor: Tolkien, J. R. R.Peso: 0.44I.S.B.N.: 9788578277109Altura: 20.000000Largura: 13.000000Profundidade: 1.000000Número de Páginas: 328Idioma: PortuguêsAcabamento: BrochuraNúmero da edição: 7Ano da edição: 2013'

In [0]:
import nltk
nltk.download('stopwords')

In [0]:
# 1 - Contagem de termos com unigrama
from sklearn.feature_extraction.text import CountVectorizer

stops = nltk.corpus.stopwords.words('portuguese')

#vect = CountVectorizer()
#vect = CountVectorizer(ngram_range=(1,1))
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df.texto)
text_vect = vect.transform(df.texto)

In [0]:
# 2 - TF-IDF com unigrama - referencia
from sklearn.feature_extraction.text import TfidfVectorizer

stops = nltk.corpus.stopwords.words('portuguese')

#vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops)
vect.fit(df.texto)
text_vect = vect.transform(df.texto)

In [0]:
# 3 - TF-IDF com unigrama e aplicando o stemização RSLP
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

df['tokens'] = df.texto.apply(word_tokenize)

from nltk.stem.rslp import RSLPStemmer
import nltk
nltk.download('rslp')

rslp = RSLPStemmer()

def stem_pandas(line):
  return ' '.join([rslp.stem(token) for token in line])

df['stemmer'] = df.tokens.apply(stem_pandas)

0       o hobbit - 7ª ed . 2013 produt novobilb bols é...
1       livr - it a cois - stephen king produt novodur...
2       box as crôn de gel e fog pocket 5 livr produt ...
3       box harry pott produt nov e físic a séri harry...
4       livr orig - dan brown produt novod ond vi ? pa...
                              ...                        
4073    red dead redemption ediç do ano goty xbox 360 ...
4074    jog gta 5 grand theft aut v ps4 míd cd + map p...
4075    zeld : breath of the wild - expansion pas - dl...
4078    gta san andre hd remastered ps3 envi imediat n...
4079    min gam nov portátil 10mil jog play mp3 mp4 mp...
Name: stemmer, Length: 2916, dtype: object

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

stops = nltk.corpus.stopwords.words('portuguese')

#vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops)
#vect = TfidfVectorizer(ngram_range=(1,2), use_idf=False, stop_words=stops)

vect.fit(df.stemmer)
text_vect = vect.transform(df.stemmer)

<2916x26528 sparse matrix of type '<class 'numpy.float64'>'
	with 265469 stored elements in Compressed Sparse Row format>

In [0]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(
    text_vect, 
    df["categoria"],
    test_size = 0.2, 
    random_state = 42
)

In [0]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

#tree.score(X_train, y_train)
tree.score(X_test, y_test)

0.9623287671232876

Exemplo 1 - 0.9554794520547946

Exemplo 2 - 0.9537671232876712

Exemplo 3 - 0.9434931506849316

Exemplo 4 - 0.9503424657534246

Exemplo 5 - 0.9623287671232876

0.9212328767123288

In [0]:
from sklearn.metrics import accuracy_score

y_prediction = tree.predict(X_test)

accuracy = accuracy_score(y_prediction, y_test)

print(accuracy)

0.9623287671232876


In [0]:
texto = """
Maleta Exclusiva Ruby Rose Profissional Completa. 
"""

texto_vetor = vect.transform([texto])

<1x35310 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [0]:
tree.predict(texto_vetor)

array(['maquiagem'], dtype=object)